In [34]:
# Import libraries
import json
import sys
sys.path.append("..")
from utils.filesystem import *
from utils.salesforce import *

In [35]:
# Config file path
config_file = '../config-salesforce.json'

# Load the config file
with open(config_file) as f:
    config = json.load(f)

# Load the basic config parameters
username = config['salesforce_username']
password = config['salesforce_password']
security_token = config["salesforce_token"]

# Load file config
input_folder = "."
input_file = config['bulk_import_file']

In [36]:
# Ignore UserWarning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np

# Load a specific worksheet from the Excel file
try:
    dataframe = load_csv_to_dataframe (input_folder, input_file, 0)

    # Replace inf and -inf with the maximum and minimum representable finite float numbers
    dataframe.replace([np.inf, -np.inf], [np.finfo(np.float64).max, np.finfo(np.float64).min], inplace=True)

    # Replace NaN with a default value (e.g., 0 or None)
    dataframe.fillna("", inplace=True)

    # Count the number of rows and columns
    rows, columns = dataframe.shape
    print(f"The dataframe has {rows} rows and {columns} columns")
    print(dataframe)

except Exception as e:
    print(f"An error occurred: {e}")

The dataframe has 3 rows and 6 columns
    Name     Date__c  Amount__c Category__c Contact__c PersonaRett__c
0  Test1  2024-04-01      23.00  Donaciones                          
1  Test2  2024-03-02     -20.42      Gastos                          
2  Test3  2024-02-03      55.00      Cuotas                          


C:\Users\perocha\AppData\Local\Temp\ipykernel_31604\1685558817.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe.fillna("", inplace=True)


In [37]:
# Create a SalesForceAPI object
sf_api = SalesForceAPI(username, password, security_token)

# Authenticate and export contacts to CSV
if sf_api.authenticate():
    print("Authentication successful")

    # Bulk import the data into Salesforce
    records = dataframe.to_dict(orient='records')
    print(f"Importing {len(records)} records into Salesforce")

    result = sf_api.bulk_import_transactions(records)
    print(result)
else:
    print("Authentication failed")

Authentication successful
Importing 3 records into Salesforce
Uploaded 3 transactions
True
